# bio-transformers

https://pypi.org/project/bio-transformers/


```
conda create --name bio-transformers python=3.7 -y && conda activate bio-transformers
or
pip install bio-transformers
```

Note that you might need to install more packages to get CUDA support (`torch.cuda.is_available()` >> True)


* This seems to work (from https://pytorch.org/get-started/locally/)


`pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html`

* I don't remember if I needed this or not but keeping it here just in case:
`pip install setuptools==59.5.0`

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
torch.cuda.is_available()

True

In [44]:
import numpy as np
import pandas as pd
from biotransformers import BioTransformers
BioTransformers.list_backend()

Use backend in this list :

  *   esm1_t34_670M_UR100
  *   esm1_t12_85M_UR50S
  *   esm1_t6_43M_UR50S
  *   esm1b_t33_650M_UR50S
  *   esm_msa1_t12_100M_UR50S
  *   esm_msa1b_t12_100M_UR50S
  *   esm1v_t33_650M_UR90S_1
  *   protbert
  *   protbert_bfd


# Load protein sequences

In [4]:
from Bio import SeqIO

# Set the input FASTA file name
fasta_file = "combined.fasta"

# Create an empty list to store the protein sequences
protein_ids = []
protein_sequences = []

# Iterate over the sequences in the FASTA file
for record in SeqIO.parse(fasta_file, "fasta"):
    # Append the ID and sequence as a tuple to the protein_sequences list
    protein_ids.append(record.id)
    protein_sequences.append(str(record.seq))
    
# Generate a list of all residues with protein name, residue ID and index
residue_names = []
for prot_ix, seq in enumerate(protein_sequences):
    protein_name = protein_ids[prot_ix]
    for aa_ix, aa in enumerate(seq):
        residue_names.append("{}_{}{}".format(protein_name, aa, aa_ix+1))

In [29]:
bio_trans = BioTransformers(backend="protbert", num_gpus=1)

# Probably should use a smaller batch_size (16) 
embeddings = bio_trans.compute_embeddings(protein_sequences, pool_mode=('cls','mean', 'full'), batch_size=32)

res_emb = pd.DataFrame(np.vstack(embeddings["full"]))
res_emb.index = residue_names
res_emb.to_pickle("combined.protbert.res.pkl")

mean_df = pd.DataFrame(embeddings["mean"])
mean_df.index = protein_ids
mean_df.to_pickle("combined.protbert.mean.pkl")

cls_df = pd.DataFrame(embeddings["cls"])
cls_df.index = protein_ids
cls_df.to_pickle("combined.protbert.cls.pkl")

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [05:34<00:00,  1.85s/it]


In [75]:
bio_trans = BioTransformers(backend="esm1b_t33_650M_UR50S", num_gpus=1)
embeddings = bio_trans.compute_embeddings(protein_sequences, pool_mode=('cls','mean', 'full'), batch_size=32)

res_emb = pd.DataFrame(np.vstack(embeddings["full"]))
res_emb.index = residue_names
res_emb.to_pickle("combined.esm1b.res.pkl")

mean_df = pd.DataFrame(embeddings["mean"])
mean_df.index = protein_ids
mean_df.to_pickle("combined.esm1b.mean.pkl")

cls_df = pd.DataFrame(embeddings["cls"])
cls_df.index = protein_ids
cls_df.to_pickle("combined.esm1b.cls.pkl")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [08:19<00:00,  2.76s/it]


###### Time different batch sizes

A batch size of 32 seems to work faster, but when I tried setting it to 64 I quickly ran out of GPU memory and it failed to comlete the task. Most of the sequences in this batch of protein sequences are very short so might need to adjust as needed. I'm going to run it at a `batch_size=16` to be safe.

In [7]:
%timeit -n 1 -r 3 bio_trans.compute_embeddings(protein_sequences[:500], pool_mode=('cls','mean'), batch_size=8)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:19<00:00,  3.29it/s]

20.1 s ± 597 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [8]:
%timeit -n 1 -r 3 bio_trans.compute_embeddings(protein_sequences[:500], pool_mode=('cls','mean'), batch_size=16)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:17<00:00,  1.83it/s]


17.9 s ± 92.1 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [9]:
%timeit -n 1 -r 3 bio_trans.compute_embeddings(protein_sequences[:500], pool_mode=('cls','mean'), batch_size=32)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:12<00:00,  1.24it/s]

13.5 s ± 294 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [15]:
from biotransformers import BioTransformers
bio_trans = BioTransformers(backend="esm1b_t33_650M_UR50S", num_gpus=1)

In [12]:
%timeit -n 1 -r 3 bio_trans.compute_embeddings(protein_sequences[:500], pool_mode=('cls','mean'), batch_size=8)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:31<00:00,  2.01it/s]


31.6 s ± 197 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [13]:
%timeit -n 1 -r 3 bio_trans.compute_embeddings(protein_sequences[:500], pool_mode=('cls','mean'), batch_size=16)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:27<00:00,  1.16it/s]

27.7 s ± 85.3 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [14]:
%timeit -n 1 -r 3 bio_trans.compute_embeddings(protein_sequences[:500], pool_mode=('cls','mean'), batch_size=32)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:21<00:00,  1.33s/it]

21.6 s ± 321 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
